In [1]:
# Importing necessary modules

import sklearn.datasets as datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
import pandas as pd 
# read text file into pandas DataFrame and 
# create header 
import os
Column_names = ["TIME_str", "TIME_sec", "latitude", "longitude", "ALTITUDE_FT", "GROUNDTRACK_DEG", \
            "GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN", "Vertical_label",\
           "Horizontal_label", "Speed_label", "Turn_rate_deg_per_sec", "REL_HEADING", "DIST_TO_DEST", "MANEUVER_ID" ]
Selected_col_names = ["GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN",  \
                      "DIST_TO_DEST"]
Target_category_name = "Vertical_label"
data_name = "HOLD_3"
cwd = os.getcwd()
mypath = cwd + f"/../data/{data_name}/"
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [3]:
count = 0

for file_name in onlyfiles:
    if "Traj" in file_name:
        if count == 0:
            file_path = mypath + file_name
            df = pd.read_csv(file_path, sep=",", header=None)
            df.columns = Column_names
            X_df = df[Selected_col_names]
            Y_target = df[Target_category_name]
        else:
            file_path = mypath + file_name
            df_temp = pd.read_csv(file_path, sep=",", header=None)
            df_temp.columns = Column_names
            
            X_df_temp = df_temp[Selected_col_names]
            Y_target_temp = df_temp[Target_category_name]
            X_df = pd.concat([X_df, X_df_temp])
            Y_target = pd.concat([Y_target, Y_target_temp])
        count +=1
        # if count >= 10:
        #     break

In [4]:
X_df["GROUNDSPEED_KTS"].max()

547.0

In [5]:
X_df["GROUNDSPEED_KTS"].min()

0.0

In [6]:
target_names =  list(set(list(Y_target)))
word_map_to_num = set(list(Y_target))
print("category:")
word_map_to_num


category:


{'  Over 250 kts under 10000ft',
 ' Climb_Nominal',
 ' Descend_Nominal',
 ' MaintainCurrentAltitude',
 ' No Conclusion: Vertical'}

In [7]:
n_sample = len(X_df)
print("number of points:", n_sample)
X_df = X_df.head(n_sample)
Y_target = Y_target.head(n_sample)
Y = []
for y in Y_target:
    for idx, s in enumerate(word_map_to_num):
        if s == y:
            Y.append(idx)
Y = np.array(Y)

number of points: 54570


In [8]:
import pandas as pd 
import dtreeviz
from sklearn.tree import export_text
# read text file into pandas DataFrame and 
# create header 
import os
cwd = os.getcwd()

# file_name_test = "Traj_AAL80-11121269.dat" # worst
file_name_test = "Traj_AAL245-11147798_L_HOLD.dat" # normal

file_path_test = mypath + file_name_test


# file_name_test = "Traj_AAL1252-11172342POS_HOLD.dat"

# file_path_test = cwd + "/../data/HOLD/" + file_name_test


df_test = pd.read_csv(file_path_test, sep=",", header=None)
df_test.columns = Column_names
X_df_test = df_test[Selected_col_names]

In [9]:
print("number of testing points:", len(X_df_test))

number of testing points: 3407


In [10]:
Z_target = df_test[Target_category_name]
n_sample = len(X_df_test)
X_df_test = X_df_test.head(n_sample)
Z_target = Z_target.head(n_sample)
Z = []
for z in Z_target:
    for idx, s in enumerate(word_map_to_num):
        if s == z:
            Z.append(idx)
Z = np.array(Z)
len(Z)

3407

## target 1: vertical_label

# Entropy

In [11]:
# Entropy, small tree with train data

criterion = "entropy"
max_depth = 8
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
viz_model = dtreeviz.model(dtree_small_entropy, X_df, Y,
                target_name="target",
                feature_names=list(X_df.columns),
                class_names=target_names)

v = viz_model.view()     # render as SVG into internal object 
v                 # pop up window
v.save(f"{data_name}_all_{Target_category_name}_{criterion}_without_xy_" + f"max_depth:{max_depth}"  + ".svg")  # optionally save as svg

0.8870075132856882


/Users/binshuaiwang/PycharmProjects/XAI_project/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names


In [12]:
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

|--- VERTICALSPEED_FT_PER_MIN <= -160.00
|   |--- VERTICALSPEED_FT_PER_MIN <= -672.00
|   |   |--- DIST_TO_DEST <= 96.90
|   |   |   |--- GROUNDSPEED_KTS <= 282.50
|   |   |   |   |--- GROUNDSPEED_KTS <= 174.50
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -736.00
|   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -736.00
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 163.00
|   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |--- GROUNDSPEED_KTS >  163.00
|   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |--- GROUNDSPEED_KTS >  174.50
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -928.00
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 235.50
|   |   |   |   |   |   |   |--- DIST_TO_DEST <= 4.02
|   |   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |   |--- DIST_TO_DEST >  4.02
|   |   |   |   |   |   |   |   |--- class:  Descend_Nomin

In [13]:
# Entropy, small tree with train data
from sklearn.tree import export_text
from sklearn.tree import export_text_in_conditions

criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
viz_model = dtreeviz.model(dtree_small_entropy, X_df, Y,
                target_name="target",
                feature_names=list(X_df.columns),
                class_names=target_names)

v = viz_model.view()     # render as SVG into internal object 
v                 # pop up window
v.save(f"{data_name}_all_{Target_category_name}_{criterion}_without_xy_" + f"max_depth:{max_depth}"  + ".svg")  # optionally save as svg
r = export_text_in_conditions(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)

0.8870075132856882


/Users/binshuaiwang/PycharmProjects/XAI_project/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names


If VERTICALSPEED_FT_PER_MIN <= -160.00 && VERTICALSPEED_FT_PER_MIN <= -672.00 && DIST_TO_DEST <= 96.90 && GROUNDSPEED_KTS <= 282.50 && GROUNDSPEED_KTS <= 174.50 && VERTICALSPEED_FT_PER_MIN <= -736.00, then  class:  Descend_Nominal

If VERTICALSPEED_FT_PER_MIN <= -160.00 && VERTICALSPEED_FT_PER_MIN <= -672.00 && DIST_TO_DEST <= 96.90 && GROUNDSPEED_KTS <= 282.50 && GROUNDSPEED_KTS <= 174.50 && VERTICALSPEED_FT_PER_MIN > -736.00 && GROUNDSPEED_KTS <= 163.00, then  class:  Descend_Nominal

If VERTICALSPEED_FT_PER_MIN <= -160.00 && VERTICALSPEED_FT_PER_MIN <= -672.00 && DIST_TO_DEST <= 96.90 && GROUNDSPEED_KTS <= 282.50 && GROUNDSPEED_KTS <= 174.50 && VERTICALSPEED_FT_PER_MIN > -736.00 && GROUNDSPEED_KTS > 163.00, then  class:  Descend_Nominal

If VERTICALSPEED_FT_PER_MIN <= -160.00 && VERTICALSPEED_FT_PER_MIN <= -672.00 && DIST_TO_DEST <= 96.90 && GROUNDSPEED_KTS <= 282.50 && GROUNDSPEED_KTS > 174.50 && VERTICALSPEED_FT_PER_MIN <= -928.00 && GROUNDSPEED_KTS <= 235.50 && DIST_TO_DEST <= 4.

In [14]:
vocabulary = {
    "GROUNDSPEED_KTS": ["very low", "low", "normal", "high", "very high"],
    "VERTICALSPEED_FT_PER_MIN": ["very low", "low", "normal", "high", "very high"],
    "Turn_rate_deg_per_sec": ["very low", "low", "normal", "high", "very high"],
    "REL_HEADING": ["extremely left", "left", "straight", "right", "extremely right"],
    "DIST_TO_DEST": ["extremely close", "close", "a little far", "far", "extremely far"]
}
# five words needs 4 landmarks
# seperate equally

def turn_threshold_to_word(X_df, name, threshold):
    # print(type(threshold))
    x_min = X_df[name].min()
    x_max = X_df[name].max()
    diff = x_max+0.0001 - x_min
    division = 5
    for i in range(division):
        if x_min + i/division*diff <= threshold < x_min + (i+1)/division*diff:
            return vocabulary[name][i]


In [123]:
from sklearn.tree import export_text_in_words
from sklearn.utils.validation import check_array , check_is_fitted
from sklearn.base import is_classifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, _criterion, _tree
import matplotlib.pyplot as plt
def export_text_in_words(
        X_df,
        decision_tree,
        *,
        feature_names=None,
        class_names=None,
        max_depth=10,
        spacing=3,
        decimals=2,
        show_weights=False,
        show_prob = True,
):

    import copy
    threshold_collection = dict()
    if feature_names is not None:
        feature_names = check_array(
            feature_names, ensure_2d=False, dtype=None, ensure_min_samples=0
        )
    if class_names is not None:
        class_names = check_array(
            class_names, ensure_2d=False, dtype=None, ensure_min_samples=0
        )

    check_is_fitted(decision_tree)
    tree_ = decision_tree.tree_
    if is_classifier(decision_tree):
        if class_names is None:
            class_names = decision_tree.classes_
        elif len(class_names) != len(decision_tree.classes_):
            raise ValueError(
                "When `class_names` is an array, it should contain as"
                " many items as `decision_tree.classes_`. Got"
                f" {len(class_names)} while the tree was fitted with"
                f" {len(decision_tree.classes_)} classes."
            )
    right_child_fmt = "{} {} is {}"
    left_child_fmt = "{} {} is {}"
    truncation_fmt = "{} {}\n"

    if feature_names is not None and len(feature_names) != tree_.n_features:
        raise ValueError(
            "feature_names must contain %d elements, got %d"
            % (tree_.n_features, len(feature_names))
        )

    if isinstance(decision_tree, DecisionTreeClassifier):
        value_fmt = "{}{} weights: {}\n"
        if not show_weights:
            value_fmt = "{}{}{}\n"
    else:
        value_fmt = "{}{} value: {}\n"

    if feature_names is not None:
        feature_names_ = [
            feature_names[i] if i != _tree.TREE_UNDEFINED else None
            for i in tree_.feature
        ]
    else:
        feature_names_ = ["feature_{}".format(i) for i in tree_.feature]

    export_text.report = ""

    def _add_leaf(value, class_name, indent, lower_bound_dict, upper_bound_dict, printable):
        ## estiblish upper and lower dict for the features, upper collection the upper value, right child of the threshold, keep decreasing
        # lower_bound_dict collect the lower value, left child of the threshold, keep increasing
        #     right_child_fmt = "{} {} <= {}" left and right children are not corresponding to the graph, this is only for the tree,
        #     left_child_fmt = "{} {} > {}" the tree is build from right small to left big
        val = ""
        is_classification = isinstance(decision_tree, DecisionTreeClassifier)
        if show_weights or not is_classification:
            val = ["{1:.{0}f}, ".format(decimals, v) for v in value]
            val = "[" + "".join(val)[:-2] + "]"
        if is_classification:
            val += " class: " + str(class_name)
        export_text.report += value_fmt.format(indent, "", val)

        ##
        if printable:
            output_conditions = []
            all_feature_names = set(list(lower_bound_dict.keys()) + list(upper_bound_dict.keys()))
            for key in all_feature_names:
                cur_vocabulary = vocabulary[key]
                interval_length = interval_length_collection[key]
                cur_list = threshold_collection[key]
                cur_words = []
                if (key in lower_bound_dict.keys()) and (key in upper_bound_dict.keys()):
                    for idx in range(len(cur_vocabulary)-1):
                        left_range, right_range = cur_list[interval_length*idx], cur_list[interval_length*(idx+1)]
                        if left_range <= lower_bound_dict[key] < right_range or left_range < upper_bound_dict[key] <= right_range:
                            cur_words.append(cur_vocabulary[idx])
                    right_range = cur_list[interval_length*(idx+1)]
                    if lower_bound_dict[key] > right_range or upper_bound_dict[key] > right_range:
                        cur_words.append(cur_vocabulary[-1])
                    cur_range_txt = f"(around {lower_bound_dict[key]:.02f} ~ {upper_bound_dict[key]:.02f})"
                    cur_condition = f"{key} is " + " or ".join(cur_words) + cur_range_txt
                    output_conditions.append(cur_condition)
                    
                elif (key in lower_bound_dict.keys()):
                    for idx in range(len(cur_vocabulary)-1):
                        left_range, right_range = cur_list[interval_length*idx], cur_list[interval_length*(idx+1)]
                        if left_range <= lower_bound_dict[key] < right_range:
                            cur_words.append(cur_vocabulary[idx])
                    right_range = cur_list[interval_length*(idx+1)]
                    if lower_bound_dict[key] > right_range:
                        cur_words.append(cur_vocabulary[-1])
                    cur_range_txt = f"(> {lower_bound_dict[key]:.02f})"
                    cur_condition = f"{key} is " + " or ".join(cur_words) + cur_range_txt
                    output_conditions.append(cur_condition)

                elif (key in upper_bound_dict.keys()):
                    for idx in range(len(cur_vocabulary)-1):
                        left_range, right_range = cur_list[interval_length*idx], cur_list[interval_length*(idx+1)]
                        if left_range < upper_bound_dict[key] <= right_range:
                            cur_words.append(cur_vocabulary[idx])
                    right_range = cur_list[interval_length*(idx+1)]
                    if upper_bound_dict[key] > right_range:
                        cur_words.append(cur_vocabulary[-1])
                    cur_range_txt = f"(<= {upper_bound_dict[key]:.02f})"
                    cur_condition = f"{key} is " + " or ".join(cur_words) + cur_range_txt
                    output_conditions.append(cur_condition)
            prob = np.max(value)/np.sum(value)*100
            output_result = f"then with {int(np.max(value))} samples support in {prob:.02f}% probability, the class is{str(class_name)}."
            print("If " + ", and ".join(output_conditions) + ", ")
            print(output_result)
            print()
    def print_tree_recurse(node, depth, lower_bound_dict, upper_bound_dict, printable):
        indent = ""

        value = None
        if tree_.n_outputs == 1:
            value = tree_.value[node][0]
        else:
            value = tree_.value[node].T[0]
        class_name = np.argmax(value)

        if tree_.n_classes[0] != 1 and tree_.n_outputs == 1:
            class_name = class_names[class_name]

        if depth <= max_depth + 1:
            info_fmt = ""
            info_fmt_left = info_fmt
            info_fmt_right = info_fmt

            if tree_.feature[node] != _tree.TREE_UNDEFINED:
                name = feature_names_[node] # the name of the feature in string
                threshold = tree_.threshold[node] # the value of the threshold
                # threshold = "{1:.{0}f}".format(decimals, threshold)
                if not printable:
                    if name not in threshold_collection:
                        threshold_collection[name] = []
                    else:
                        threshold_collection[name].append(threshold)
                export_text.report += right_child_fmt.format(indent, name, threshold) # output a line for the right child
                export_text.report += info_fmt_left

                # print_tree_recurse(tree_.children_left[node], depth + 1, cur_pre_text + right_child_fmt.format(indent, name, threshold) )
                # print the less than case, right_child_fmt is corresponding to the <= case.
                changed_upper_bound_dict = copy.deepcopy(upper_bound_dict)
                changed_upper_bound_dict[name]= threshold
                print_tree_recurse(tree_.children_left[node], depth + 1, lower_bound_dict, changed_upper_bound_dict, printable)

                export_text.report += left_child_fmt.format(indent, name, threshold)
                export_text.report += info_fmt_right
                changed_lower_bound_dict = copy.deepcopy(lower_bound_dict)
                changed_lower_bound_dict[name]=threshold
                print_tree_recurse(tree_.children_right[node], depth + 1, changed_lower_bound_dict, upper_bound_dict, printable)
            else:  # leaf
                _add_leaf(value, class_name, indent, lower_bound_dict, upper_bound_dict, printable)
        else:
            subtree_depth = _compute_depth(tree_, node)
            if subtree_depth == 1:
                _add_leaf(value, class_name, indent)
            else:
                trunc_report = "truncated branch of depth %d" % subtree_depth
                export_text.report += truncation_fmt.format(indent, trunc_report)

    upper_bound_dict = dict()
    lower_bound_dict = dict()
    # construct the threshold_collection
    print_tree_recurse(0, 1, lower_bound_dict, upper_bound_dict, printable = False)

    # print the range of levels for each feature
    interval_length_collection = dict()
    for feature_name in Selected_col_names:
        threshold_collection[feature_name].sort()
        cur_list = threshold_collection[feature_name]
        cur_vocal = vocabulary[feature_name]
        interval_length = (len(cur_list)-1) // (len(cur_vocal)-1)
        interval_length_collection[feature_name] = interval_length
        # threshold_collection[0:intervals_length] is the first level
        # threshold_collection[intervals_length:2*intervals_length] is the second level
        # ...
        print(f"###########################################")
        print(f"{r'feature name':^20} : {feature_name}")
        print(f"{r'number of ranges':^20} : {len(cur_vocal)}")
        print(f"-------------------------------------------")
        for i in range(len(cur_vocal)-1):
            left_range = cur_list[interval_length*i]
            right_range = cur_list[interval_length*(i+1)]
            
            print(f"{cur_vocal[i]:^20} : [{left_range:.2f},{right_range:.2f})")
        right_range = cur_list[interval_length*(i+1)]
        print(f"{cur_vocal[-1]:^20} :  >= {right_range:.2f}")
    print(f"###########################################")

    
    upper_bound_dict = dict()
    lower_bound_dict = dict()
    print_tree_recurse(0, 1, lower_bound_dict, upper_bound_dict, printable = True)

    # print(threshold_collection)
    return export_text.report

In [124]:

r = export_text_in_words(X_df, dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=True, show_prob = True)

###########################################
    feature name     : GROUNDSPEED_KTS
  number of ranges   : 5
-------------------------------------------
      very low       : [0.50,235.50)
        low          : [235.50,342.50)
       normal        : [342.50,430.00)
        high         : [430.00,478.50)
     very high       :  >= 478.50
###########################################
    feature name     : VERTICALSPEED_FT_PER_MIN
  number of ranges   : 5
-------------------------------------------
      very low       : [-1824.00,-928.00)
        low          : [-928.00,-672.00)
       normal        : [-672.00,-96.00)
        high         : [-96.00,288.00)
     very high       :  >= 288.00
###########################################
    feature name     : DIST_TO_DEST
  number of ranges   : 5
-------------------------------------------
  extremely close    : [0.00,13.58)
       close         : [13.58,52.03)
    a little far     : [52.03,148.89)
        far          : [148.89,391.94)
   e

In [ ]:
import graphviz
from sklearn.tree import export_graphviz
dot_data = export_graphviz(X_df, dtree_small_entropy,
                           feature_names=X_df.columns,  
                           class_names = target_names,
                           filled=False,  
                           rounded = True,
                            precision = 2,
                           proportion=False)
graph = graphviz.Source(dot_data)
graph.format = "png"
graph.render(f"{data_name}__all_{Target_category_name}_{criterion}" + "importances")

In [ ]:
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)

In [ ]:
print(r)

In [ ]:
tree = model.tree_

In [ ]:
print(tree.children_left[tree.children_left[2]])

In [ ]:
criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=10, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

In [ ]:
# Entropy, large tree with train data

criterion = "entropy"
dtree_large_n = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_n.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_n, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

In [ ]:
# Entropy, large tree with test data

criterion = "entropy"
dtree_large_entropy = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

In [ ]:
# Entropy, small tree with test data

criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=10, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

In [ ]:
# Entropy, large rf with test data

criterion = "entropy"
num_of_trees = 3
rf_small_entropy = RandomForestClassifier(n_estimators=num_of_trees)
model=rf_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

In [ ]:
# Entropy, small rf with test data

criterion = "entropy"
num_of_trees = 3
rf_small_entropy = RandomForestClassifier(n_estimators=num_of_trees, max_depth=5, min_samples_leaf=20)
model=rf_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

# Gini

In [ ]:
# Gini, small tree with train data

criterion = "gini"
dtree_small_gini = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_gini.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

In [ ]:
# Gini, large tree with train data
criterion = "gini"
dtree_large_gini = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_gini.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

In [ ]:
# Gini, large tree with test data

criterion = "gini"
dtree_large_gini = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_gini.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

In [ ]:
# Gini, small tree with test data

criterion = "gini"
dtree_small_gini = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_gini.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

## Log_loss

In [ ]:
# Log_loss, small tree with train data

criterion = "log_loss"
dtree_small_Log_loss = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_Log_loss, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

In [ ]:
# Log_loss, large tree with train data
criterion = "log_loss"
dtree_large_Log_loss = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

In [ ]:
# Log_loss, large tree with test data
criterion = "log_loss"
dtree_large_Log_loss = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

In [ ]:
# Log_loss, small tree with test data

criterion = "log_loss"
dtree_small_Log_loss = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)